# UC7 Demonstration

© Crown-owned copyright 2025, Defence Science and Technology Laboratory UK


## Network Description

<p align="center">
    <a href="./_package_data/uc7/uc7_network_detailed_svg.svg" target="_blank">
        <img src="./_package_data/uc7/uc7_network_detailed_svg.svg" alt="Image">
    </a>  
    
</p>


In [ ]:
!primaite setup

In [ ]:
from primaite.session.environment import PrimaiteGymEnv
from primaite.simulator.network.hardware.nodes.host.computer import Computer
from primaite.simulator.network.hardware.nodes.host.server import Server
from primaite.simulator.network.hardware.nodes.network.router import Router
from primaite.simulator.network.hardware.nodes.network.switch import Switch
from primaite.simulator.network.container import Network
from primaite.simulator.network.hardware.nodes.network.firewall import Firewall
from primaite.game.game import PrimaiteGame
from primaite.simulator.sim_container import Simulation
import yaml
from pprint import pprint
from primaite.config.load import load, _EXAMPLE_CFG

In [ ]:
use_case_7_config = load(_EXAMPLE_CFG/"uc7_config.yaml")
env = PrimaiteGymEnv(env_config=use_case_7_config)

In [ ]:
uc7_network = env.game.simulation.network
uc7_network.show()

## UC7 Network | Home Office Subnet

In [ ]:
home_pub_pc_1: Computer = env.game.simulation.network.get_node_by_hostname("HOME-PUB-PC-1")
home_pub_pc_2: Computer = env.game.simulation.network.get_node_by_hostname("HOME-PUB-PC-2")
home_pub_pc_srv: Server = env.game.simulation.network.get_node_by_hostname("HOME-PUB-SRV")
home_pub_rt_dr: Router = env.game.simulation.network.get_node_by_hostname("HOME-PUB-RT-DR")

In [ ]:
# HOME PC 1 --> HOME PC 2
home_pub_pc_1.ping(home_pub_pc_2.network_interface[1].ip_address)

In [ ]:
# HOME PC 2 --> HOME SERVER
home_pub_pc_2.ping(home_pub_pc_srv.network_interface[1].ip_address)

In [ ]:
# HOME SERVER --> HOME ROUTER
home_pub_pc_srv.ping(home_pub_rt_dr.network_interface[1].ip_address)

## UC7 Network | Internet

In [ ]:
isp_pub_rt_br: Router = env.game.simulation.network.get_node_by_hostname("ISP-PUB-RT-BR")
isp_pub_srv_dns: Server = env.game.simulation.network.get_node_by_hostname("ISP-PUB-SRV-DNS")

In [ ]:
home_pub_pc_1.ping(isp_pub_srv_dns.network_interface[1].ip_address)

In [ ]:
# HOME ROUTER --> INTERNET ROUTER
home_pub_rt_dr.ping(isp_pub_rt_br.network_interface[1].ip_address)

In [ ]:
home_pub_rt_dr.route_table.show()

In [ ]:
# HOME ROUTER --> INTERNET ISP
home_pub_rt_dr.ping(isp_pub_srv_dns.network_interface[1].ip_address)

In [ ]:
# INTERNET ROUTER --> INTERNET DNS
isp_pub_rt_br.ping(target_ip_address=isp_pub_srv_dns.network_interface[1].ip_address)

In [ ]:
# HOME ROUTER --> INTERNET DNS
home_pub_rt_dr.ping(target_ip_address=isp_pub_srv_dns.network_interface[1].ip_address)

In [ ]:
# PC 1 --> INTERNET DNS

home_pub_pc_1.ping(target_ip_address=isp_pub_srv_dns.network_interface[1].ip_address)

In [ ]:
isp_pub_rt_br.show_nic()
isp_pub_rt_br.ping("192.168.1.2")


## UC7 Network | Remote Site Subnet

In [ ]:
network: Network = env.game.simulation.network


rem_pub_fw: Firewall = network.get_node_by_hostname(hostname="REM-PUB-FW")
rem_pub_rt_dr: Router = network.get_node_by_hostname(hostname="REM-PUB-RT-DR")
rem_pub_sw_as: Switch = network.get_node_by_hostname(hostname="REM-PUB-SW-AS")
rem_pub_pc_1: Computer = network.get_node_by_hostname(hostname="REM-PUB-PC-1")
rem_pub_pc_2: Computer = network.get_node_by_hostname(hostname="REM-PUB-PC-2")
rem_pub_srv: Computer = network.get_node_by_hostname(hostname="REM-PUB-SRV")

In [ ]:
# Remote PC 1 --> Remote PC_2

rem_pub_pc_1.ping(rem_pub_pc_2.network_interface[1].ip_address)

In [ ]:
# Remote PC 2 --> Remote Server

rem_pub_pc_2.ping(rem_pub_srv.network_interface[1].ip_address)

In [ ]:
# Remote Server --> Remote Site Router

rem_pub_srv.ping(rem_pub_rt_dr.network_interface[1].ip_address)

In [ ]:
# Remote Site Router --> Remote Pub Firewall (Internal)

rem_pub_rt_dr.ping(rem_pub_fw.network_interface[1].ip_address)

In [ ]:
# Remote Site Router --> Remote Pub Firewall (External)

rem_pub_rt_dr.ping(rem_pub_fw.network_interface[2].ip_address)

In [ ]:
# Remote Pub Firewall (external) --> Public Internet

rem_pub_fw.ping(isp_pub_rt_br.network_interface[1].ip_address)

In [ ]:
# Remote Site PC 1 --> Public DNS

rem_pub_pc_1.ping(target_ip_address=isp_pub_srv_dns.network_interface[1].ip_address)

In [ ]:
# Remote Site PC 1 --> Home Office PC 1

rem_pub_pc_1.ping(target_ip_address=home_pub_pc_1.network_interface[1].ip_address)

## UC7 Network | SOME_TECH_DMZ

In [ ]:
st_pub_fw: Firewall = network.get_node_by_hostname(hostname="ST-PUB-FW")
st_dmz_pub_srv_web: Server = network.get_node_by_hostname(hostname="ST-DMZ-PUB-SRV-WEB")

In [ ]:
# Some Tech Firewall --> Some Tech DMZ public web server

st_pub_fw.ping(st_dmz_pub_srv_web.network_interface[1].ip_address)

In [ ]:
# Some Tech Firewall --> Public DNS Server

st_pub_fw.ping(isp_pub_srv_dns.network_interface[1].ip_address)

In [ ]:
# Some Tech DMZ public web serv

In [ ]:
# Some Tech Firewall --> Internet Router (Port 4)

st_pub_fw.ping(isp_pub_rt_br.network_interface[4].ip_address)

## UC7 Network | SOME_TECH_INTRANET

In [ ]:
st_intra_prv_rt_cr: Router = network.get_node_by_hostname(hostname="ST-INTRA-PRV-RT-CR")
st_intra_prv_rt_dr_1: Router = network.get_node_by_hostname(hostname="ST-INTRA-PRV-RT-DR-1")
st_intra_prv_rt_dr_2: Router = network.get_node_by_hostname(hostname="ST-INTRA-PRV-RT-DR-2")

In [ ]:
# Some Tech Intranet Private Router CR --> Some Tech Public Firewall

st_intra_prv_rt_cr.ping(st_pub_fw.network_interface[3].ip_address)

In [ ]:
# Some tech intranet router DR 1 --> Some Tech Intranet Router CR

st_intra_prv_rt_dr_1.ping(st_intra_prv_rt_cr.network_interface[1].ip_address)

In [ ]:
# Some tech intranet router DR 2 --> Some Tech Intranet Router CR

st_intra_prv_rt_dr_2.ping(st_intra_prv_rt_cr.network_interface[1].ip_address)

In [ ]:
# Some tech intranet router DR 1 --> Some tech intranet router DR 2

st_intra_prv_rt_dr_1.ping(st_intra_prv_rt_dr_2.network_interface[1].ip_address)

In [ ]:
# Some tech intranet router DR 1 --> Some Tech Firewall (Internal Port)

st_intra_prv_rt_dr_1.ping(st_pub_fw.network_interface[3].ip_address)

In [ ]:
# Some tech intranet router DR 1 --> Some Tech Firewall (DMZ Port)

st_intra_prv_rt_dr_1.ping(st_pub_fw.network_interface[2].ip_address)

In [ ]:
# Some tech intranet router DR 1 --> Some Tech Firewall (External Port)

st_intra_prv_rt_dr_1.ping(st_pub_fw.network_interface[1].ip_address)

In [ ]:
# Some Tech Intranet Router DR 1 --> Some Tech DMZ web-server

st_intra_prv_rt_dr_1.ping(target_ip_address=st_dmz_pub_srv_web.network_interface[1].ip_address)

In [ ]:
# Some tech intranet router DR 1 --> Public Internet

st_intra_prv_rt_dr_1.ping(isp_pub_rt_br.network_interface[3].ip_address)

In [ ]:
# Some tech intranet router DR 1 --> Public DNS 

st_intra_prv_rt_dr_1.ping(isp_pub_srv_dns.network_interface[1].ip_address)

In [ ]:
# Some Tech Intranet Router DR 1 --> Home Office PC 1

st_intra_prv_rt_dr_1.ping(target_ip_address=home_pub_pc_1.network_interface[1].ip_address)

In [ ]:
# Some Tech Intranet Router DR 1 --> Remote Site PC 1

st_intra_prv_rt_dr_1.ping(target_ip_address=rem_pub_pc_1.network_interface[1].ip_address)

## UC7 Network | Some Tech Head Office

In [ ]:
st_head_office_private_pc_1: Computer = network.get_node_by_hostname("ST-HO-PRV-PC-1")
st_head_office_private_pc_2: Computer = network.get_node_by_hostname("ST-HO-PRV-PC-2")
st_head_office_private_pc_3: Computer = network.get_node_by_hostname("ST-HO-PRV-PC-3")

In [ ]:
# ST Home office PC 1 --> ST Home Office PC 2

st_head_office_private_pc_1.ping(st_head_office_private_pc_1.network_interface[1].ip_address)

In [ ]:
# ST Home office PC 3 --> ST Router DR 2  

st_head_office_private_pc_1.ping(st_intra_prv_rt_dr_2.network_interface[1].ip_address)

In [ ]:
# ST Home office PC 1 --> Public DNS

st_head_office_private_pc_1.ping(isp_pub_srv_dns.network_interface[1].ip_address)

## UC7 Network | Some Tech Human Resources

In [ ]:
st_human_resources_private_pc_1: Computer = network.get_node_by_hostname("ST-HR-PRV-PC-1")
st_human_resources_private_pc_2: Computer = network.get_node_by_hostname("ST-HR-PRV-PC-2")
st_human_resources_private_pc_3: Computer = network.get_node_by_hostname("ST-HR-PRV-PC-3")

In [ ]:
# ST Human Resources PC 1 --> ST Human Resources PC 2 

st_human_resources_private_pc_1.ping(st_human_resources_private_pc_2.network_interface[1].ip_address)

In [ ]:
# ST Human Resources PC 2 --> # ST Human Resources PC 3

st_human_resources_private_pc_2.ping(st_human_resources_private_pc_3.network_interface[1].ip_address)

In [ ]:
# ST Human Resources PC 1 --> ST Intranet Router DR 2 

st_human_resources_private_pc_1.ping(st_intra_prv_rt_dr_2.network_interface[1].ip_address)

In [ ]:
# ST Human Resources PC 1 --> Public DNS 

st_human_resources_private_pc_1.ping(isp_pub_srv_dns.network_interface[1].ip_address)

## UC7 Network | Some Tech Data (Database / Database Backup)

In [ ]:
st_data_private_server_database: Server = network.get_node_by_hostname("ST-DATA-PRV-SRV-DB")
st_data_private_server_storage: Server = network.get_node_by_hostname("ST-DATA-PRV-SRV-STORAGE")

In [ ]:
# ST Data Private Database --> ST Data Private Storage

st_data_private_server_database.ping(st_data_private_server_storage.network_interface[1].ip_address)

In [ ]:
# ST Data Private Storage --> Public DNS

st_data_private_server_storage.ping(isp_pub_srv_dns.network_interface[1].ip_address)

## UC7 Network | Some Tech Project A 

In [ ]:
st_project_a_private_pc_1: Computer = network.get_node_by_hostname("ST-PROJ-A-PRV-PC-1")
st_project_a_private_pc_2: Computer = network.get_node_by_hostname("ST-PROJ-A-PRV-PC-2")
st_project_a_private_pc_3: Computer = network.get_node_by_hostname("ST-PROJ-A-PRV-PC-3")

In [ ]:
# ST Private Project A PC 1 --> ST Private Project A PC 2

st_project_a_private_pc_1.ping(st_project_a_private_pc_2.network_interface[1].ip_address)

In [ ]:
# ST Private Project A PC 2 --> ST Private Project A PC 3

st_project_a_private_pc_3.ping(st_project_a_private_pc_3.network_interface[1].ip_address)

In [ ]:
# ST Private Project A PC 3 --> Public DNS

st_project_a_private_pc_3.ping(isp_pub_srv_dns.network_interface[1].ip_address)

In [ ]:
# ST Private Project A PC 1 --> ST-INTRA-PRV-RT-CR

st_project_a_private_pc_1.ping(st_intra_prv_rt_cr.network_interface[1].ip_address)

In [ ]:
# ST Private Project A PC 1 --> ST-INTRA-PRV-RT-DR-2

st_project_a_private_pc_1.ping("192.168.170.2")

## UC7 Network | Some Tech Project B

In [ ]:
st_project_b_private_pc_1: Computer = network.get_node_by_hostname("ST-PROJ-B-PRV-PC-1")
st_project_b_private_pc_2: Computer = network.get_node_by_hostname("ST-PROJ-B-PRV-PC-2")
st_project_b_private_pc_3: Computer = network.get_node_by_hostname("ST-PROJ-B-PRV-PC-3")

In [ ]:
# ST Private Project B PC 1 --> ST Private Project B PC 2

st_project_b_private_pc_1.ping(st_project_b_private_pc_2.network_interface[1].ip_address)

In [ ]:
# ST Private Project B PC 2 --> ST Private Project B PC 3

st_project_b_private_pc_2.ping(st_project_b_private_pc_3.network_interface[1].ip_address)

In [ ]:
# ST Private Project B PC 3 --> Public DNS

st_project_b_private_pc_3.ping(isp_pub_srv_dns.network_interface[1].ip_address)

## UC7 Network | Some Tech Project C

In [ ]:
st_project_c_private_pc_1: Computer = network.get_node_by_hostname("ST-PROJ-C-PRV-PC-1")
st_project_c_private_pc_2: Computer = network.get_node_by_hostname("ST-PROJ-C-PRV-PC-2")
st_project_c_private_pc_3: Computer = network.get_node_by_hostname("ST-PROJ-C-PRV-PC-3")

In [ ]:
# ST Private Project C PC 1 --> ST Private Project C PC 2

st_project_c_private_pc_1.ping(st_project_c_private_pc_2.network_interface[1].ip_address)

In [ ]:
# ST Private Project C PC 2 --> ST Private Project C PC 3

st_project_c_private_pc_2.ping(st_project_c_private_pc_3.network_interface[1].ip_address)

In [ ]:
# ST Private Project c PC 3 --> Public DNS

st_project_c_private_pc_3.ping(isp_pub_srv_dns.network_interface[1].ip_address)

In [ ]:
st_project_c_private_pc_1.ping(st_project_b_private_pc_1.network_interface[1].ip_address)

In [ ]:
st_project_b_private_pc_1.ping(st_project_a_private_pc_1.network_interface[1].ip_address)

In [ ]:
st_project_a_private_pc_1.ping(st_head_office_private_pc_1.network_interface[1].ip_address)

In [ ]:
st_project_a_private_pc_1.ping(st_human_resources_private_pc_1.network_interface[1].ip_address)

## UC7 Network | Applications & Services

In [ ]:
# DNS Server

isp_pub_srv_dns.software_manager.show()

In [ ]:
# Web Server & Web Browser

st_project_a_web_browser = st_project_a_private_pc_1.software_manager.software["web-browser"]
st_project_a_web_browser.get_webpage()

In [ ]:
st_web_server = st_dmz_pub_srv_web.software_manager.software["web-server"]
st_web_server.sys_log.show()

In [ ]:
st_database_client = st_project_a_private_pc_1.software_manager.software["database-client"]
st_database_client.connect()

st_database = st_data_private_server_database.software_manager.software["database-service"]
st_database.sys_log.show()

In [ ]:
st_ftp_server = st_data_private_server_storage.software_manager.software["ftp-server"]

st_ftp_server.sys_log.show()

In [ ]:
from primaite.simulator.system.services.dns.dns_client import DNSClient
home_pub_rt_dr.acl.show()

home_pub_pc_1: Computer = network.get_node_by_hostname("HOME-PUB-PC-1")
dns_client: DNSClient = home_pub_pc_1.software_manager.software["dns-client"]

dns_client.check_domain_exists(target_domain="some_tech.com")
dns_client.dns_cache.get("some_tech.com", None) is not None
len(dns_client.dns_cache) == 1


In [ ]:
env.step(0)

In [ ]:
# from primaite.simulator.system.applications.application import ApplicationOperatingState
# from primaite.simulator.system.applications.database_client import database-client
# from primaite.simulator.system.services.service import ServiceOperatingState

# home_pub_pc_1_dns_client: dns-client = home_pub_pc_1.software_manager.software["dns-client"]
# home_pub_pc_1_database_client: database-client = home_pub_pc_1.software_manager.software["database-client"]


# home_pub_pc_1_dns_client.operating_state = ServiceOperatingState.PAUSED
# home_pub_pc_1_database_client.operating_state = ApplicationOperatingState.CLOSED

In [ ]:
env.step(0)

In [ ]:
for _ in range(80):
    env.step(0)

In [ ]:
def print_agent_actions_except_do_nothing(agent_name):
    print(f"\n{agent_name} actions this episode:")
    for item in env.game.agents[agent_name].history:
        if item.action != "DONOTHING":
            node, application = 'unknown', 'unknown'
            if (node_id := item.parameters.get('node_id')) is not None:
                node = env.game.agents[agent_name].action_manager.node_names[node_id]
            if (application_id := item.parameters.get('application_id')) is not None:
                application = env.game.agents[agent_name].action_manager.application_names[node_id][application_id]
            print(f"Step: {item.timestep}, action: {item.action}, {node}, {application}, response: {item.response.status}")

print_agent_actions_except_do_nothing("HOME_WORKER-1-DB")

In [ ]:
green_agent = env.game.agents.get("HOME_WORKER-1-DB")
green_agent.reward_function.total_reward

In [ ]:
print_agent_actions_except_do_nothing("HOME_WORKER-1-WEB")

In [ ]:
green_agent = env.game.agents.get("HOME_WORKER-1-WEB")
green_agent.reward_function.total_reward

In [ ]:
red_agent = env.game.agents.get("attacker")
red_agent.logger.show()

In [ ]:
st_project_a_private_pc_1.file_system.show()

# # st_project_a_private_pc_1.file_system.folders["exfiltration"].show()

# st_project_a_private_pc_1.software_manager.show()

In [ ]:
st_intra_prv_rt_cr.acl.show()